In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run Mask-Utils.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from pathlib import Path

from tqdm import trange

In [4]:
TEXTURE_SIZE = 400

In [5]:
face_boxes, tddfa = load_models()

In [6]:
predictor = get_vertices_predictor(face_boxes, tddfa, is_cropped=True)

In [7]:
FACECOVER_PATHS = list(sorted(str(p) for p in Path('masks/').glob('*.png')))
len(FACECOVER_PATHS)

20

In [8]:
FACECOVER_IMAGES = [cv2.imread(p, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
                    for p in FACECOVER_PATHS]

In [9]:
def blur_facecover(facecover):
    facecover = facecover.copy()
    
    solid = facecover[..., :3]
    alpha = facecover[..., -1]

    for c in range(3):
        solid[..., c][alpha < 0.1] = np.mean(solid[..., c][alpha > 0.1])
    
    facecover[..., :3] = blur_multichannel(facecover[..., :3], sigma=2.5, mode='nearest')
    facecover[..., -1:] = blur_multichannel(facecover[..., -1:], sigma=2.5, mode='constant')
    return facecover

In [10]:
FACECOVER_TEXTURES = [blur_facecover(
        FacecoverTextureWarper()(FACECOVER_IMAGES[i % len(FACECOVER_IMAGES)], TEXTURE_SIZE)
    ) for i in trange(100)]

100%|██████████| 100/100 [01:04<00:00,  1.54it/s]


In [11]:
cached_linear_nd_creator = CachedInterpolatorCreator(interpolate.LinearNDInterpolator)
cached_linear_nd_ext_creator = CachedInterpolatorCreator(LinearNDInterpolatorExt)

In [12]:
face_indicator = None
convexer = None
blur_mix_weights = None

___

In [13]:
from skimage.io import imread
from random import choice
from tqdm import tqdm

paths = Path('~/datasets/lfw/lfw_mtcnnpy_160').expanduser().glob('**/*_0*.*')

for i, path in tqdm(list(enumerate(paths))):
    image_path = str(path)

    new_image_path = image_path.replace("_0", "_1")
    if Path(new_image_path).is_file():
        continue
    
    image = cv2.imread(image_path)
    if image is None:
        continue
    
    ver = predictor(image)
    texture = uv_tex(image, [ver], tddfa.tri, uv_h=TEXTURE_SIZE, uv_w=TEXTURE_SIZE)

    uv_coords = get_uv_coords(TEXTURE_SIZE)
    uv2ver_sparse = get_sparse_uv2ver(uv_coords, TEXTURE_SIZE, ver)
    uv2ver = inpaint(uv2ver_sparse,
                     mask=np.isnan(uv2ver_sparse).astype(np.uint8),
                     interpolator_cls=cached_linear_nd_creator)

    if face_indicator is None:
        face_indicator = get_face_indicator(uv2ver)
    if convexer is None:
        convexer = FixedConvexer(ver, uv_coords, face_indicator, uv2ver)
    
    convex_uv2ver_sparse = convexer(uv2ver)
    convex_uv2ver = inpaint(convex_uv2ver_sparse,
                            mask=np.isnan(convex_uv2ver_sparse).astype(np.uint8),
                            interpolator_cls=cached_linear_nd_ext_creator)

    if blur_mix_weights is None:
        blur_mix_weights = get_blur_weights(convex_uv2ver.shape)[..., None]
    
    blurred_convex_uv2ver = blur_multichannel(convex_uv2ver, sigma=(2.5, 5.0), mode='nearest')
    convex_uv2ver = convex_uv2ver * (1.0 - blur_mix_weights) + blurred_convex_uv2ver * blur_mix_weights
    
    convex_mesh = get_mesh(convex_uv2ver, face_indicator, TEXTURE_SIZE, TEXTURE_SIZE)
    
    visible_skin = get_visible_skin(uv2ver)
    target_lightness_ratio = get_cheeks_lightness_ratio(texture, visible_skin)
    light_x = find_light_x_position(
        target_lightness_ratio,
        uv_coords, ver, tddfa.tri, texture, visible_skin,
        render_app, image.shape,
    )

    facecover = FACECOVER_TEXTURES[i % len(FACECOVER_TEXTURES)].copy()
    facecover_tr = facecover_color_transfer(facecover, texture)
    update_colors(convex_mesh, facecover_tr)

    ambient_w, direct_w = get_lightning_params(target_lightness_ratio, light_x, facecover_tr)
    
    final_render = render_facecover(
        render_app, convex_mesh, image, light_x, ambient_w, direct_w, return_intermediate=False
    )     
    cv2.imwrite(new_image_path, final_render)

100%|██████████| 13234/13234 [12:19<00:00, 17.89it/s]  
